
## 🏋️‍♂️ Personalized Health, Workout & Diet Planning System

## 📌 Overview
This notebook builds an intelligent health assistant that collects user details, calculates essential health metrics, and generates personalized **7-day workout and diet plans** using Large Language Models (LLMs) and LangChain.

## 🔍 What This Notebook Does
The notebook performs the following tasks step by step:

1. **Collects User Inputs**
   - Height, weight, age
   - Gender
   - Activity level
   - Weight goal
   - Fitness goal

2. **Calculates Health Metrics**
   - Body Mass Index (BMI) and BMI category
   - Basal Metabolic Rate (BMR)
   - Daily maintenance calories adjusted for activity level
   - Calorie target based on weight goal (lose, gain, maintain)

3. **Uses an AI Agent**
   - Automatically decides when to call tools
   - Computes health metrics using a dedicated function
   - Returns structured, machine-readable output

4. **Generates Personalized Plans**
   - Creates a **7-day workout plan**
   - Creates a **7-day diet plan**
   - Uses prompt templates for consistency and clarity

5. **Runs Chains in Parallel**
   - Executes workout and diet plan generation simultaneously
   - Uses `RunnableParallel` for efficient execution

## ⚙️ Technologies Used
- Python
- LangChain
- Prompt Templates
- RunnableParallel
- LLMs (Google Gemini / compatible model)

## ✅ Final Output
At the end of this notebook, the user receives:
- Accurate health metrics
- A structured 7-day workout plan
- A structured 7-day diet plan

All outputs are personalized based on the user’s goals and activity level.

---

🎯 **Goal:** To demonstrate how agents, tools, and parallel chains can be combined to build a real-world, AI-powered health planning system.
l

#Milestone 1 : Colab setup - packages and API key

In [2]:
!pip install -q langchain
!pip install -q langchain-google-genai
!pip install -q google-generativeai
!pip install -q langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.3/53.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 713.3/713.3 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.1/489.1 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.9/234.9 kB 15.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.43.0, but you have google-auth 2.47.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 2.9 MB/s eta 0:00:00
ERROR: pip

In [3]:
from google.colab import userdata
from langchain_core.tools import tool
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import create_agent
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableParallel
from langchain_core.output_parsers import StrOutputParser
import json

In [4]:
GEMINI_API_KEY=userdata.get('GEMINI_API_KEY_2')

#Milestone 2 : Create BMI Calculation Tool

In [5]:
@tool
def generate_BMI(weight:float,height:float)->str:
  """This function generates the body mass index by taking the input weight and height """
  BMI=weight/(height*height)
  category=""
  if(BMI<18.5):
    category="Underweight"
  elif(BMI>=18.5 and BMI<=24.9):
    category="Normal"
  elif(BMI>=25 and BMI<=29.9):
    category="Overweight"
  else:
    category="Obese"
  return f"BMI : {BMI} , Category : {category}"



In [12]:
generate_BMI.invoke({"weight":48,"height":1.74})

'BMI : 15.854141894569956 , Category : Underweight'

#Milestone 3 : Complete Health Metrics Tool

In [6]:
@tool
def generate_complete_BMI(
    height: float,
    weight: float,
    age: int,
    gender: str,
    activity_level: str,
    weight_goal: str
) -> dict:

    """
    Calculates BMI, BMR, and maintenance calories based on user details.

    Parameters:
    height (float): Height in meters
    weight (float): Weight in kilograms
    age (int): Age in years
    gender (str): 'male' or 'female'
    activity_level (str): sedentary, lightly active, moderately active, very active, extra active
    weight_goal (str): lose, gain, or maintain

    Returns:
    str: Formatted string containing BMI, category, BMR, and maintenance calories
    """


    BMI = weight / (height ** 2)


    if(BMI<18.5):
      category="Underweight"
    elif(BMI>=18.5 and BMI<=24.9):
      category="Normal"
    elif(BMI>=25 and BMI<=29.9):
      category="Overweight"
    else:
      category="Obese"

    # Convert height to cm for BMR
    height_cm = height * 100

    # BMR Calculation (Mifflin-St Jeor)
    gender = gender.lower().strip()
    if gender == "male":
        bmr = (10 * weight) + (6.25 * height_cm) - (5 * age) + 5
    else:
        bmr = (10 * weight) + (6.25 * height_cm) - (5 * age) - 161

    # Activity Factors
    activity_factors = {
        "sedentary": 1.2,
        "lightly active": 1.375,
        "moderately active": 1.55,
        "very active": 1.725,
        "extra active": 1.9
    }

    activity_level = activity_level.lower().strip()
    maintenance_calories = bmr * activity_factors[activity_level]

    # Adjust for weight goal
    weight_goal = weight_goal.lower().strip()
    if "lose" in weight_goal:
        maintenance_calories -= 500
        goal_text = "lose weight"
    elif "gain" in weight_goal:
        maintenance_calories += 500
        goal_text = "gain weight"
    else:
        goal_text = "maintain weight"

    return {
        "BMI":round(BMI,2),
        "category":category,
        "BMR":int(bmr),
        "maintenance_calaories":int(maintenance_calories)
    }


#Milestorne 4 : Collect user input and initialize llm

In [7]:
def collect_user_health_data():

    height = float(input("Enter your height in meters: "))
    weight = float(input("Enter your weight in kg: "))
    age = int(input("Enter your age in years: "))

    gender = input("Enter your gender (male/female): ").lower().strip()

    activity_level = input(
        "Enter your activity level (sedentary / lightly active / moderately active / very active / extra active): "
    ).lower().strip()

    weight_goal = input("Enter your weight goal (lose / gain / maintain): ").lower().lower().strip()
    fitness_goal = input("Enter your fitness goal: ").lower().strip()

    return {
        "height": height,
        "weight": weight,
        "age": age,
        "gender": gender,
        "activity_level": activity_level,
        "weight_goal": weight_goal,
        "fitness_goal": fitness_goal
    }


In [17]:
user_data=collect_user_health_data()


Enter your height in meters: 1.74
Enter your weight in kg: 48
Enter your age in years: 21
Enter your gender (male/female): male
Enter your activity level (sedentary / lightly active / moderately active / very active / extra active): moderately active
Enter your weight goal (lose / gain / maintain): gain
Enter your fitness goal: weight gain
{'height': 1.74, 'weight': 48.0, 'age': 21, 'gender': 'male', 'activity_level': 'moderately active', 'weight_goal': 'gain', 'fitness_goal': 'weight gain'}


In [19]:
from pprint import pprint

pprint(user_data)

{'activity_level': 'moderately active',
 'age': 21,
 'fitness_goal': 'weight gain',
 'gender': 'male',
 'height': 1.74,
 'weight': 48.0,
 'weight_goal': 'gain'}


In [8]:
llm=ChatGoogleGenerativeAI(
    api_key=GEMINI_API_KEY,
    model="gemini-2.5-flash-lite"
)

In [9]:
response=llm.invoke("hello how are you")
print(response.content)

Hello! I am a large language model, trained by Google. I am functioning well and ready to assist you. How can I help you today?


#Milestone 5 : Create Agent and Get Health Metrics

In [10]:
agent = create_agent(
    model=llm,
    tools=[generate_complete_BMI],
    system_prompt="""You are an AI assistant with access to tools.
Whenever a task requires calculation, data processing, or structured output, you must use the appropriate tool instead of answering directly.
Use tools only when necessary and rely on them for accurate results.
 """
    )

In [11]:
agent_input = {
    "messages": [{
        "role": "user",
        "content": "Calculate health metrics for a male with height 1.75 meters, weight 70 kg, age 30 years, activity level moderately_active, and weight goal lose_weight."
    }]
}

In [12]:
response=agent.invoke(agent_input)


In [13]:
print(response["messages"][-1].content)

The user's BMI is 22.86, which falls into the "Normal" category. His Basal Metabolic Rate (BMR) is 1648 calories, and his maintenance calories are 2055 calories.


In [14]:
print(response["messages"][-2].content)

{"BMI": 22.86, "category": "Normal", "BMR": 1648, "maintenance_calaories": 2055}


#Milestone 6 : Generate Workout and Diet Plans

In [15]:
workout_plan_prompt = PromptTemplate(
    template="""
You are a fitness planning assistant.

Using the following health metrics and goals, create a personalized 7-day workout plan.

Health Metrics:
- BMI: {bmi} ({bmi_category})
- BMR: {bmr} calories/day
- Maintenance Calories: {maintenance_calories} calories/day
- Activity Level: {activity_level}
- Weight Goal: {weight_goal}
- Fitness Goal: {fitness_goal}

Instructions:
- Create a 7-day workout plan (Day 1 to Day 7)
- Match intensity to the activity level
- Align exercises with the weight goal and fitness goal
- Include rest or recovery days if appropriate
- Use clear headings and bullet points
- Keep exercises simple and equipment-optional

Format:
Day 1: <Workout Focus>
- Exercise: sets x reps or duration
- Exercise: sets x reps or duration

Day 2: ...
""",
        input_variables=[
        "bmi",
        "bmi_category",
        "bmr",
        "maintenance_calories",
        "weight_goal",
        "activity_level",
        "fitness_goal"
    ]
)


diet_plan_prompt = PromptTemplate(
    template="""
You are a nutrition planning assistant.

Using the following health metrics and goals, create a personalized 7-day diet plan.

Health Metrics:
- BMI: {bmi} ({bmi_category})
- BMR: {bmr} calories/day
- Maintenance Calories: {maintenance_calories} calories/day
- Activity Level: {activity_level}
- Weight Goal: {weight_goal}
- Fitness Goal: {fitness_goal}

Instructions:
- Create a 7-day diet plan (Day 1 to Day 7)
- Adjust calorie intake according to the weight goal
- Ensure meals are balanced and realistic
- Avoid extreme diets
- Use simple food items
- Display approximate calories per meal

Format:
Day 1:
- Breakfast: <meal> (<calories>)
- Lunch: <meal> (<calories>)
- Dinner: <meal> (<calories>)
- Snacks: <meal> (<calories>)

Day 2: ...
""",
    input_variables=[
        "bmi",
        "bmi_category",
        "bmr",
        "maintenance_calories",
        "weight_goal",
        "activity_level",
        "fitness_goal"
    ]
)


In [16]:
workout_plan_chain = workout_plan_prompt | llm | StrOutputParser()
diet_plan_chain = diet_plan_prompt | llm | StrOutputParser()

parallel_chain = RunnableParallel(
    {
        "workout_plan":workout_plan_chain,
         "diet_plan":diet_plan_chain
    }
)

In [17]:
input_data = {
    "bmi": 22.86,
    "bmi_category": "Normal",
    "bmr": 1649,
    "maintenance_calories": 2056,
    "weight_goal": "lose",
    "activity_level": "moderately active",
    "fitness_goal": "build muscle and get fit"
}
results = parallel_chain.invoke(input_data)
print("Workout Plan:\n")
print(results["workout_plan"])

print("\nDiet Plan:\n")
print(results["diet_plan"])


Workout Plan:

Here is a personalized 7-day workout plan designed to help you build muscle and get fit, while also supporting your weight loss goal. This plan considers your moderately active lifestyle and aims for a balanced approach with progressive overload for muscle building.

**Important Considerations Before You Start:**

*   **Listen to Your Body:** This is a template. If you feel excessive pain, stop. Modify exercises as needed.
*   **Nutrition is Key:** To lose weight and build muscle simultaneously, you'll need to be in a slight caloric deficit while ensuring adequate protein intake. Focus on whole, unprocessed foods.
*   **Progressive Overload:** To continue building muscle, gradually increase the weight, reps, or sets over time.
*   **Warm-up & Cool-down:** Always start each workout with 5-10 minutes of light cardio (e.g., jogging in place, jumping jacks) and dynamic stretching. End with 5-10 minutes of static stretching.
*   **Hydration:** Drink plenty of water throughout

#Milestone 7 : Display Output

In [34]:
user_data=collect_user_health_data()


agent_input = {
    "messages": [{
        "role": "user",
        "content": f"Calculate health metrics for a male with height {user_data["height"]} meters, weight {user_data["weight"]} kg, age {user_data["age"]} years, activity level {user_data["activity_level"]}, and weight goal {user_data["weight_goal"]}."
    }]
}


health_metrics=agent.invoke(agent_input)


health_metrics_dict=json.loads(health_metrics["messages"][-2].content)


input_data = {
    "bmi": health_metrics_dict["BMI"],
    "bmi_category": health_metrics_dict["category"],
    "bmr": health_metrics_dict["BMR"],
    "maintenance_calories": health_metrics_dict["maintenance_calaories"],
    "weight_goal": user_data["weight_goal"],
    "activity_level": user_data["activity_level"],
    "fitness_goal": user_data["fitness_goal"]
}


results = parallel_chain.invoke(input_data)

print("""============================================================
HEALTH METRICS
============================================================\n\n""")
print(health_metrics["messages"][-1].content)
print("""\n\n============================================================
7-DAY WORKOUT PLAN
============================================================\n\n\n\n\n""")
print(results["workout_plan"])
print("""\n\n\n\n\n============================================================
7-DAY DIET PLAN
============================================================\n\n\n\n\n""")
print(results["diet_plan"])
print("""\n\n\n\n\n============================================================
YOUR PERSONALIZED FITNESS PLAN IS READY!
============================================================""")

Enter your height in meters: 1.74
Enter your weight in kg: 48
Enter your age in years: 21
Enter your gender (male/female): male
Enter your activity level (sedentary / lightly active / moderately active / very active / extra active): lightly active
Enter your weight goal (lose / gain / maintain): gain
Enter your fitness goal: weight gain
HEALTH METRICS


The user's BMI is 15.85, which falls into the "Underweight" category. His Basal Metabolic Rate (BMR) is 1467 calories, and his estimated daily maintenance calories are 2517.


7-DAY WORKOUT PLAN





Here is a personalized 7-day workout plan designed to support your weight gain and fitness goals, considering your current health metrics. The focus will be on building muscle mass and strength, which will contribute to healthy weight gain.

**Important Considerations:**

*   **Nutrition is Key:** For weight gain, especially when underweight, **nutrition is paramount**. This workout plan is designed to stimulate muscle growth, but you **mus